In [28]:
import pandas as pd
import optuna


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, confusion_matrix,fbeta_score


In [40]:
X_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_train.csv", header=None)
X_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_test.csv", header=None)
y_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_train.csv", header=None)
y_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_test.csv", header=None)



print("X train ",X_train.shape)
print("X test",X_test.shape)
print("Y_Train", y_train.shape)
print("Y_test", y_test.shape)

X train  (4050, 1000)
X test (375, 1000)
Y_Train (4050, 1)
Y_test (375, 1)


In [41]:
print((y_train.iloc[:,0].value_counts() / y_train.shape[0]) * 100)


0
0    62.493827
1    37.506173
Name: count, dtype: float64


Here, the balanced option automatically calculates weights as:


Weight for Class i= N /
2×N 
i
​
 
 
Where 
𝑁
N is the total number of samples, and 
𝑁
𝑖
N 
i
​
  is the number of samples in class 
𝑖
i.

In [47]:

def objective(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e2)
    max_iter = trial.suggest_int('max_iter', 50, 500)
    solver = trial.suggest_categorical('solver', ['liblinear', 'lbfgs', 'saga'])
    penalty = trial.suggest_categorical('penalty', ['l2'])

    # Create and train the Logistic Regression model
    model = LogisticRegression(C=C, max_iter=max_iter, solver=solver, penalty=penalty, random_state=42,
                               class_weight="balanced")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    precision = precision_score(y_test, y_pred)

    return precision

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
print(f"Best hyperparameters: {study.best_params}")
print(f"Best precision: {study.best_value}")


# Ran for: 3m 7.8s

[I 2024-12-02 00:44:32,750] A new study created in memory with name: no-name-8673b649-ea6f-48bb-8a5f-8a2c8b9046dc
/tmp/ipykernel_105009/1051339289.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e2)
/home/pavithra/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-12-02 00:44:33,025] Trial 0 finished with value: 0.2777777777777778 and parameters: {'C': 2.519452626289445e-05, 'max_iter': 300, 'solver': 'liblinear', 'penalty': 'l2'}. Best is trial 0 with value: 0.2777777777777778.
/tmp/ipykernel_105009/1051339289.py:2: FutureWarning: suggest_loguniform has been deprecated in

Best hyperparameters: {'C': 33.48602800121877, 'max_iter': 365, 'solver': 'liblinear', 'penalty': 'l2'}
Best precision: 0.75


Best hyperparameters: {'C': 0.006739078892927555, 'max_iter': 483, 'solver': 'liblinear', 'penalty': 'l2'}
Best accuracy: 0.9051490514905149

In [48]:
# logistic regression with l2 regularization.
param = {'C': 0.006739078892927555, 'max_iter': 483, 'solver': 'liblinear', 'penalty': 'l2'}
#param = {'C': 0.24921835861380356, 'max_iter': 317, 'solver': 'saga', 'penalty': 'l2'}
param = {'C': 33.48602800121877, 'max_iter': 365, 'solver': 'liblinear', 'penalty': 'l2'}
model = LogisticRegression(**param)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

/home/pavithra/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:

def classification_metrics(y_test, y_pred):
    print("="*50)
    print("The accuracy score", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1 score:", f1_score(y_test, y_pred))
    print("="*50)
    print(classification_report(y_test, y_pred))
    print("\n")
    print("="*50)
    print("confusion Matrix")
    print(confusion_matrix(y_test, y_pred))
    print("="*50)


classification_metrics(y_test, y_pred)

The accuracy score 0.904
Precision: 0.75
Recall: 0.07894736842105263
F1 score: 0.14285714285714285
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       337
           1       0.75      0.08      0.14        38

    accuracy                           0.90       375
   macro avg       0.83      0.54      0.55       375
weighted avg       0.89      0.90      0.87       375



confusion Matrix
[[336   1]
 [ 35   3]]


In [35]:
y_test.value_counts()

0    337
1     38
Name: count, dtype: int64

In [36]:
y_test.value_counts() / y_test.shape[0]

0    0.898667
1    0.101333
Name: count, dtype: float64

In [20]:
d = pd.DataFrame(y_pred)
d.value_counts()

0    362
1     13
Name: count, dtype: int64

In [7]:
model.predict(X_test)

ValueError: feature_names mismatch: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '1